# Algoritmos de ordenación

Ordenar colecciones de datos es una tarea recurrente en la informática, pues facilita y hace más eficientes las búsquedas, comparaciones y el procesamiento de los datos.

In [1]:
sorted(['García', 'Rodríguez', 'González', 'Fernández', 'López', 'Martínez', 'Sánchez'])

['Fernández',
 'García',
 'González',
 'López',
 'Martínez',
 'Rodríguez',
 'Sánchez']

In [2]:
help(sorted)

Help on built-in function sorted in module builtins:

sorted(iterable, /, *, key=None, reverse=False)
    Return a new list containing all items from the iterable in ascending order.
    
    A custom key function can be supplied to customize the sort order, and the
    reverse flag can be set to request the result in descending order.



Python (como cualquier lenguaje de programación decente) incluye funciones predefinidas para ordenar secuencias, el método `list.sort` y la función `sorted` que hemos usado en temas anteriores. Sin embargo, es interesante conocer algunos algoritmos de ordenación y reflexionar superficialmente sobre su eficiencia. Los algoritmos que consideraremos son válidos para ordenar cualquier tipo de datos con un operador $<$ (`__lt__`) de orden total. El siguiente resultado sobre el número mínimo de comparaciones para ordenar una secuencia arbitraria es bien conocido.

**Teorema** (Teorema 8.1, [CLRS22](https://ucm.on.worldcat.org/oclc/1319462289)): cualquier algoritmo de ordenación basado en comparaciones necesita al menos $n \cdot \log(n)$ de ellas en el caso peor para ordenar una lista de tamaño $n$.

## Ordenación de la burbuja

La metáfora del algoritmo es que los elementos flotan según su *peso* hasta el lugar que les corresponde en la lista. Esto se consigue permutando elementos contiguos en múltiples pasadas a la lista hasta que no haya dos elementos contiguos desordenados. Es poco eficiente, pues requiere $n^2$ operaciones el caso peor.

In [3]:
def ordena_burbuja_peor(v: list):
    cambiado = True
    while cambiado:
        cambiado = False
        for k in range(1, len(v)):
            if v[k-1] > v[k]:
                v[k], v[k-1] = v[k-1], v[k]
                cambiado = True
    return v

El `return v` tiene como única finalidad poder mostrar el resultado más fácilmente en las siguientes ejecuciones, pero el algoritmo modifica la lista argumento in situ.

In [4]:
ordena_burbuja_peor([3, 2, 1])

[1, 2, 3]

Si observamos que el elemento mayor de todos los considerados en una ronda se coloca siempre en la posición que le corresponde al final del vector, podemos mejorar el algoritmo parando la ronda antes de tiempo.

In [5]:
def ordena_burbuja(v: list):
    for m in range(len(v)):
        for k in range(1, len(v) - m):
            if v[k-1] > v[k]:
                v[k], v[k-1] = v[k-1], v[k]
    return v

In [6]:
ordena_burbuja([3, 2, 1])

[1, 2, 3]

Esta optimización se puede llevar más lejos observando que la lista está ordenada desde la última permutación realizada en adelante, como se deduce de la condición del `if`.

In [7]:
def ordena_burbuja_mejor(v: list):
    # Índice a partir del cual el vector está terminado
    terminado_desde = len(v)
    
    # Queremos que esté terminado desde cero o uno (es lo mismo)
    while terminado_desde > 1:
        última_permutación = 0
        for k in range(1, terminado_desde):
            if v[k-1] > v[k]:
                # Permutamos los valores
                v[k], v[k-1] = v[k-1], v[k]
                # Posición de la última permutación
                última_permutación = k
        # El vector está ordenado desde la última permutación
        terminado_desde = última_permutación
    return v

In [8]:
ordena_burbuja_mejor([3, 2, 1])

[1, 2, 3]

In [9]:
ordena_burbuja_mejor(['García', 'Rodríguez', 'González', 'Fernández', 'López', 'Martínez', 'Sánchez'])

['Fernández',
 'García',
 'González',
 'López',
 'Martínez',
 'Rodríguez',
 'Sánchez']

## Ordenación por inserción

Como sugiere su nombre, el método de ordenación por inserción consiste en insertar sucesivamente en una nueva lista los elementos de la lista original, pero en la posición que les corresponde según el orden.

La posición en el que ha de colocarse un elemento en una lista ordenada se puede averiguar con la búsqueda binaria vista en el tema de recursión. En lugar de copiar la función de allí usaremos la implementación incluida en el paquete `bisect` de la biblioteca estándar de Python.

In [10]:
from bisect import bisect  # búsqueda binaria
help(bisect)

Help on built-in function bisect_right in module _bisect:

bisect_right(a, x, lo=0, hi=None, *, key=None)
    Return the index where to insert item x in list a, assuming a is sorted.
    
    The return value i is such that all e in a[:i] have e <= x, and all e in
    a[i:] have e > x.  So if x already appears in the list, a.insert(i, x) will
    insert just after the rightmost x already there.
    
    Optional args lo (default 0) and hi (default len(a)) bound the
    slice of a to be searched.



In [11]:
def ordena_inserción(v: list) -> list:
    # Nueva lista siempre ordenada
    ordenada = []
    
    # Insertamos cada elemento de v en la posición correcta de ordenada
    for elem in v:
        # Buscamos para saber la posición correcta
        pos = bisect(ordenada, elem)

        ordenada.insert(pos, elem)
    
    return ordenada

In [12]:
ordena_inserción([3, 2, 1])

[1, 2, 3]

Este algoritmo ejecuta del orden de $n^2$ operaciones en el caso peor, pues insertar en una posición obliga a desplazar todos los elementos a su derecha, potencialmente la lista entera.

## Ordenación por mezcla

Consiste en dividir la lista en mitades, ordenar estas mitades recursivamente y mezclar ambas listas ordenadas en una lista común.

In [13]:
def ordena_mezcla(v: list):
    # La lista vacía o de un solo elemento está ordenada (caso base)
    if len(v) <= 1:
        return v

    # Parte la lista por la mitad (caso recursivo)
    mitad = len(v) // 2

    # Ordena recursivamente cada mitad de la lista
    izda = ordena_mezcla(v[:mitad])
    dcha = ordena_mezcla(v[mitad:])

    # Mezcla ambas listas
    return mezcla(izda, dcha)

La función `mezcla(v, w)` recorre las listas ordenadas `v` y `w` simultáneamente, añadiendo a la lista resultado el menor elemento de lo que queda de ambas listas. El resultado es también una lista ordenada.

In [14]:
def mezcla(v: list, w: list):
    """Mezcla un par de listas ordenadas en otra lista ordenada"""
    
    # Lista nueva que contendrá la mezcla
    mezcla_vw = []
    
    # Índices para recorrer ambas listas de entrada
    i_v, i_w = 0, 0
    
    # Recorre las listas avanzando en la del elemento menor
    while i_v < len(v) and i_w < len(w):
        if v[i_v] <= w[i_w]:
            mezcla_vw.append(v[i_v])
            i_v += 1
        else:
            mezcla_vw.append(w[i_w])
            i_w += 1
    
    # Añade los restos (solo una puede tener resto)
    mezcla_vw += v[i_v:]
    mezcla_vw += w[i_w:]
    
    return mezcla_vw

In [15]:
ordena_mezcla([3, 2, 1])

[1, 2, 3]

Este algoritmo hace $x_0 = x_1 = 1$ y $x_n = 2 \cdot n + 2 \cdot x_{n/2}$ operaciones, una cantidad que es del orden de $n \cdot \log n$.

## Ordenación estúpida (anécdota)

Haciendo honor a su nombre, consiste en barajar aleatoriamente una lista y comprobar si está ordenada hasta que acabe estando ordenada. No es un *algoritmo* propiamente dicho porque no está asegurado que termine (aunque lo hace con probabilidad 1). Hay $n!$ ordenaciones posibles, así que no es probable que converja pronto.

In [16]:
import random  # para shuffle (barajar)

In [17]:
help(random.shuffle)

Help on method shuffle in module random:

shuffle(x, random=None) method of random.Random instance
    Shuffle list x in place, and return None.
    
    Optional argument random is a 0-argument function returning a
    random float in [0.0, 1.0); if it is the default None, the
    standard random.random will be used.



In [18]:
def ordena_estúpida(v: list) -> list:
    
    while not ordenada(v):
        random.shuffle(v)
    
    return v

In [19]:
def ordenada(v: list) -> bool:
    """Comprueba si una lista está ordenada"""
    
    # o all(v[k-1] > v[k] for k in range(1, len(v)))
    for k in range(1, len(v)):
        if v[k-1] > v[k]:
            return False
    
    return True

In [20]:
ordena_estúpida([3, 2, 1])

[1, 2, 3]

## Breve comparativa

Sin ser una comparativa empírica rigurosa de los algoritmos de ordenación, podemos calcular los tiempos de ejecución de los distintos algoritmos sobre un vector aleatorio fijo.

In [21]:
lista_aleatoria = random.choices(range(0, 500), k=500)
lista_aleatoria[:8]

[231, 281, 23, 485, 14, 68, 245, 222]

Como los algoritmos modifican la lista que reciben como argumento, se hace una copia de `lista_aleatoria` antes de cada llamada. Usamos la versión de `%%timeit` con dos porcentajes para que la copia no forme parte del tiempo medido.

El método predefinido `list.sort` es bastante más rápido que nuestras funciones. Está implementado en el lenguaje C en lugar de en Python (que comparativamente no cuenta con la eficiencia entre sus numerosas virtudes).

In [22]:
%%timeit
tmp = lista_aleatoria.copy()
tmp.sort()

27.1 µs ± 887 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [23]:
%%timeit
tmp = lista_aleatoria.copy()
ordena_burbuja_peor(tmp)

31.3 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%%timeit
tmp = lista_aleatoria.copy()
ordena_burbuja_mejor(tmp)

16.8 ms ± 3.32 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%%timeit
tmp = lista_aleatoria.copy()
ordena_inserción(tmp)

125 µs ± 28.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
%%timeit
tmp = lista_aleatoria.copy()
ordena_mezcla(tmp)

1.24 ms ± 288 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


La implementación de `ordena_mezcla` utiliza la operación de tomar una sublista `v[n:m]` que genera una copia de la lista original. ¿Sería más eficiente generalizar `ordena_mezcla` con los límites de la sublista a ordenar para evitar esas copias?

In [27]:
def ordena_mezcla_lims_aux(v: list, n: int, m: int):
    # La lista vacía o de un solo elemento está ordenada (caso base)
    if m - n <= 1:
        return v[n:m]
    
    # Parte la lista por la mitad (caso recursivo)
    mitad = (n + m) // 2
    
    # Ordena recursivamente cada mitad de la lista
    izda = ordena_mezcla_lims_aux(v, n, mitad)
    dcha = ordena_mezcla_lims_aux(v, mitad, m)
    
    # Mezcla ambas listas
    return mezcla(izda, dcha)

def ordena_mezcla_lims(v: list):
    return ordena_mezcla_lims_aux(v, 0, len(v))

In [28]:
%%timeit
tmp = lista_aleatoria.copy()
ordena_mezcla_lims(tmp)

1.45 ms ± 483 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## ¿Qué algoritmo usa Python?

En la implementación oficial de Python, `sorted` y `sort` usan [Timsort](https://en.wikipedia.org/wiki/Timsort), una combinación de la ordenación por mezcla y la ordenación por inserción.

## Referencias

* §12.2 «Sorting algorithms» del [libro de Guttag](https://ucm.on.worldcat.org/oclc/1347116367) (§10.2 en la [edición de 2013](https://ucm.on.worldcat.org/oclc/1025935018)).
* §5.2 «Internal sorting» del libro [The Art of Computer Programming Vol. 3 «Sorting and Searching»](https://ucm.on.worldcat.org/oclc/1025547052) de Donald Knuth.
* §2.1 «Insertion sort» y §8.1 «Lower bounds for sorting» del [libro de Cormen, Leiserson, Rivest y Stein](https://ucm.on.worldcat.org/oclc/1319462289).
* «[*Sorting the slow way: an analysis of perversely awful randomized sorting algorithms*](https://doi.org/10.1007/978-3-540-72914-3_17)» de Gruber, Holzer y Ruepp en FUN 2007 (stupid sort).